In [39]:
import pandas as pd
import geopandas as gpd
import random
import geojson

In [3]:
taxi = gpd.read_file('data/NYC_Taxi_Zones.geojson')

In [19]:
taxi.tail()

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry,Public Transit
258,0.000168611097013,256,0.0679149669603,256,Williamsburg (South Side),Brooklyn,(POLYGON ((-73.95834207500002 40.7133063009999...,0.83
259,0.000394552487366,259,0.126750305191,259,Woodlawn/Wakefield,Bronx,(POLYGON ((-73.85107116191898 40.9103715201109...,0.83
260,0.000422345326907,260,0.133514154636,260,Woodside,Queens,(POLYGON ((-73.90175373399988 40.7607754749999...,0.83
261,0.0000343423231652,261,0.0271204563616,261,World Trade Center,Manhattan,(POLYGON ((-74.01332610899988 40.7050307879999...,0.83
262,0.000122330270966,262,0.0490636231541,262,Yorkville East,Manhattan,(POLYGON ((-73.94383256699986 40.7828590889999...,0.83


In [16]:
round(random.uniform(0,1),2)

0.11

In [17]:
round(random.uniform(0,1),2)

0.28

In [34]:
taxi['Public Transit'] = 1
taxi['Public Transit'] = taxi['Public Transit'].map(lambda x: round(random.uniform(0.1,0.5),2))
taxi['Taxi'] = 1
taxi['Taxi'] = taxi['Taxi'].map(lambda x: round(random.uniform(0.1,0.5),2))
taxi['Others'] = 1
taxi['Others'] = 1-taxi['Public Transit']-taxi['Taxi']
taxi['Top choice'] = taxi[['Public Transit', 'Taxi', 'Others']].idxmax(axis=1)

In [36]:
type(taxi)

geopandas.geodataframe.GeoDataFrame

In [37]:
taxi.head()

,shape_area,objectid,shape_leng,location_id,zone,borough,geometry,Public Transit,Taxi,Others,Top choice
0,0.0007823067885,1,0.116357453189,1,Newark Airport,EWR,(POLYGON ((-74.18445299999996 40.6949959999999...,0.44,0.23,0.33,Public Transit
1,0.00486634037837,2,0.43346966679,2,Jamaica Bay,Queens,(POLYGON ((-73.82337597260663 40.6389870471767...,0.47,0.36,0.17,Public Transit
2,0.000314414156821,3,0.0843411059012,3,Allerton/Pelham Gardens,Bronx,(POLYGON ((-73.84792614099985 40.8713422339999...,0.19,0.34,0.47,Others
3,0.000111871946192,4,0.0435665270921,4,Alphabet City,Manhattan,(POLYGON ((-73.97177410965318 40.7258212813370...,0.32,0.34,0.34,Taxi
4,0.000497957489363,5,0.0921464898574,5,Arden Heights,Staten Island,(POLYGON ((-74.17421738099989 40.5625680859999...,0.26,0.49,0.25,Taxi


In [45]:
def data2geojson(df):
    features = []
    insert_features = lambda X: features.append(
            geojson.Feature(properties=dict(objectid=X["objectid"],
                                            shape_area=X["shape_area"],
                                            shape_leng=X["shape_leng"],
                                            zone=X["zone"],
                                            borough=X["borough"],
                                            Public_Transit=X["Public Transit"],
                                            Taxi=X["Taxi"],
                                            Others=X["Others"],
                                            Top_choice=X["Top choice"]),
                            geometry=(X["geometry"])))
    df.apply(insert_features, axis=1)
    with open('NYC_transportationChoice.geojson', 'w', encoding='utf8') as fp:
        geojson.dump(geojson.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

# col = ['lat','long','elev','name','description']
# data = [[-29.9953,-70.5867,760,'A','Place ñ'],
#         [-30.1217,-70.4933,1250,'B','Place b'],
#         [-30.0953,-70.5008,1185,'C','Place c']]

# df = pd.DataFrame(data, columns=col)
data2geojson(taxi)